In [10]:
import utils

import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from torch import optim
import time
import math

In [11]:
modellist = []

In [12]:
BS = 1000

In [13]:
train_iter, val_iter, test_iter, text_field = utils.load_imdb(batch_size=BS)

Loading Data
2
1


In [14]:
def logistic_reg_torch(nfeats,nclasses):
    model = torch.nn.Sequential()
    model.add_module('logistic',torch.nn.Linear(nfeats, nclasses, bias=True))
    return model

def train(model,loss,optimizer,x,y):
    x = Variable(x)
    y = Variable(y)

    optimizer.zero_grad()

    forward_x = model.forward(x)
    obj = loss.forward(forward_x,y)

    obj.backward()

    optimizer.step()

    return obj.data[0]

def predict(model, x):
    # x = Variable(x)
    forward_x = model.forward(x)
    return forward_x.data.numpy().argmax(axis=1)

In [15]:
for eta in [0.05]:#[0,0.001,0.01,0.05,0.1,0.2,1]:
    model = logistic_reg_torch(245703,2)
    optimizer = optim.SGD(model.parameters(),lr=eta)
    loss = torch.nn.CrossEntropyLoss()

    timebefore = time.clock()

    print "TRAINING"

    for k in range(0,30):
        cost = 0.
        for batch in train_iter:
            xval = utils.bag_of_words(batch,text_field)
            yval = batch.label-1
            cost += train(model,loss,optimizer,xval.data,yval.data)
        predtrain = predict(model,xval)
        print "Epoch %d, cost = %f, acc = %.2f%%"%(k + 1, cost, 100. * np.mean(predtrain == yval.data.numpy()))

        # Validation

        for batch in val_iter:
            xval = utils.bag_of_words(batch,text_field)
            yval1 = batch.label-1
        predval = predict(model,xval)
        print "Validation acc = %.2f%%"%(100. * np.mean(predval == yval1.data.numpy()))

    print time.clock()-timebefore
    
    modelparams = (list(model.parameters())[0])

    modellist.append([eta,len(modelparams[modelparams<0.0001])/(len(modelparams)+0.0),model])

TRAINING
Epoch 1, cost = 180.054640, acc = 57.10%
Validation acc = 55.60%
Epoch 2, cost = 89.280216, acc = 49.30%
Validation acc = 50.40%
Epoch 3, cost = 148.176300, acc = 69.60%
Validation acc = 67.40%
Epoch 4, cost = 123.677407, acc = 68.40%
Validation acc = 65.60%
Epoch 5, cost = 132.049227, acc = 62.10%
Validation acc = 58.40%
Epoch 6, cost = 144.680719, acc = 73.50%
Validation acc = 70.30%
Epoch 7, cost = 102.090236, acc = 75.80%
Validation acc = 72.90%
Epoch 8, cost = 67.207338, acc = 45.00%
Validation acc = 50.00%
Epoch 9, cost = 111.468314, acc = 66.60%
Validation acc = 66.90%
Epoch 10, cost = 99.322690, acc = 72.50%
Validation acc = 73.10%
Epoch 11, cost = 49.385196, acc = 77.50%
Validation acc = 77.50%
Epoch 12, cost = 114.886445, acc = 77.90%
Validation acc = 75.10%
Epoch 13, cost = 75.806565, acc = 80.80%
Validation acc = 78.00%
Epoch 14, cost = 30.346605, acc = 76.90%
Validation acc = 77.50%
Epoch 15, cost = 89.738757, acc = 78.50%
Validation acc = 78.40%
Epoch 16, cost = 

In [86]:
for i in range(0,len(modellist)):
    model = modellist[i][2]
    timebefore = time.clock()

    print "TEST"

    for batch in test_iter:
        xval = utils.bag_of_words(batch,text_field)
        yval = batch.label-1
    predtest = predict(model,xval)
    print "acc = %.2f%%"%(100. * np.mean(predtest == yval.data.numpy()))

    print time.clock()-timebefore

TEST
acc = 52.30%
3.362704
TEST
acc = 71.70%
3.359324
TEST
acc = 81.50%
3.349769
TEST
acc = 84.20%
3.379448
TEST
acc = 81.50%
3.241962
TEST
acc = 67.60%
3.010442
TEST
acc = 59.60%
2.960639


In [85]:
for i in range(0,len(modellist)):
    print modellist[i][0]

0
0.001
0.01
0.05
0.1
0.2
1


In [83]:
for i in range(0,len(modellist)):
    print modellist[i][1]/(245703)

0.524969170096
0.525563383434
0.523713589171
0.522333874637
0.519401472509
0.511776412986
0.500862830328


In [82]:
len(modellist)

7

In [80]:
modellist[2][1]/245703

0.5237135891706654

In [29]:
params05=list(modellist[0][2].parameters())

In [34]:
params05=params05[0].data.numpy()

In [41]:
params05_0 = params05[0]
params05_1 = params05[1]

In [60]:
for i in range(0,5):
    maxval = (np.argmax(params05_1))
    minval = (np.argmin(params05_1))
    
    print "max",text_field.vocab.itos[maxval]
    print "min",text_field.vocab.itos[minval]
    
    params05_1[maxval] = 0
    params05_1[minval] = 0

max great
min bad
max best
min worst
max well
min nothing
max love
min no
max excellent
min waste
